In [ ]:
# https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.width', 5000)

In [ ]:
col_dtypes = {
    #'client_id': np.int64,
    #'loan_type': str
}
#dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d')

train = pd.read_csv("../data/house-prices/train.csv",delimiter=",")
sub = pd.read_csv("../data/house-prices/test.csv",delimiter=",")#,parse_dates=['loan_start','loan_end'], date_parser=dateparse, dtype=loans_dtypes)

In [ ]:
sub.info(memory_usage='deep')

In [ ]:
# use some automl
import h2o
h2o.init()

In [ ]:
# Load a pandas data frame to H2O
hf = h2o.H2OFrame(train)
hf_sub = h2o.H2OFrame(sub)

In [ ]:
x = hf.names
x.remove("SalePrice")
Y = "SalePrice"

In [ ]:
train, test = hf.split_frame([0.7], seed=42)

In [ ]:
from h2o.estimators.xgboost import H2OXGBoostEstimator
xgb = H2OXGBoostEstimator(nfolds=10, seed=1)
xgb.train(x=x, y=Y, training_frame=train,
         validation_frame=test)

print(xgb)

In [ ]:
# Grid Search
from h2o.grid.grid_search import H2OGridSearch

xgb_parameters = {'max_depth': [3, 4, 5, 6],
                  'sample_rate': [0.7, 0.8],
                  'col_sample_rate': [0.7, 0.8],
                  'ntrees': [200, 300, 400]}

xgb_grid_search = H2OGridSearch(model=H2OXGBoostEstimator,
                                grid_id='example_grid',
                                hyper_params=xgb_parameters)

xgb_grid_search.train(x=x,
                      y=Y,
                      training_frame=train,
                      validation_frame=test,
                      nfolds = 10,
                      learn_rate=0.01,
                      seed=42)

In [ ]:
grid_results = xgb_grid_search.get_grid(sort_by='rmse',
                                        decreasing=False)
print(grid_results)

In [ ]:
xgb = grid_results.models[0]

preds = xgb.predict(hf_sub)

In [ ]:
from h2o.automl import H2OAutoML
autoML = H2OAutoML(max_runtime_secs=3600, 
            nfolds = 10,            
            stopping_metric = 'RMSE',
            sort_metric = 'RMSE',
            exclude_algos=['DRF','GLM'],
        )

autoML.train(x=x,
             y=Y,
             training_frame=train,
             validation_frame=test,
        )

print(autoML.leaderboard)

In [ ]:
import h2o

preds = autoML.leader.predict(hf_sub)

In [ ]:
submission = hf_sub['Id']

In [ ]:
submission['SalePrice'] = preds

In [ ]:
h2o.h2o.download_csv(submission, "../data/house-prices/submission.csv")